In [12]:
import nltk

import re
import numpy as np
import pandas as pd
from nltk import RegexpTokenizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import word_tokenize, wordpunct_tokenize
from sklearn.naive_bayes import MultinomialNB
#from sklearn.linear_model import LogisticRegression
#from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report



In [2]:
# чтобы посмотреть на таблицу
df = pd.read_csv('songdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
songsdata = pd.read_csv('songdata.csv', sep=',')
artists = songsdata.groupby(['artist']).size().sort_values(ascending=False)
artists

artist
Donna Summer                                    191
Gordon Lightfoot                                189
Bob Dylan                                       188
George Strait                                   188
Reba Mcentire                                   187
Loretta Lynn                                    187
Alabama                                         187
Cher                                            187
Dean Martin                                     186
Chaka Khan                                      186
Hank Williams Jr.                               185
Neil Young                                      185
Nazareth                                        184
Cliff Richard                                   184
Indigo Girls                                    184
America                                         184
Johnny Cash                                     183
Kiss                                            183
Chris Rea                                       182
Bon J

Кажется, датасет не очень сбалансирован, поэтому я лучше возьму исполнителей с топом-количеством песен для чистоты эксперимента. Например, пусть это будут исполнители с 188 песнями и выше.

In [4]:
artists_new = artists[:6]
bands = songsdata.loc[(songsdata['artist'] == 'Donna Summer') | (songsdata['artist'] == 'Gordon Lightfoot') |
                  (songsdata['artist'] == 'Bob Dylan') | (songsdata['artist'] == 'George Strait')
            ]
artists_new
bands

,artist,song,link,text
1576,Bob Dylan,4Th Time Around,/b/bob+dylan/4th+time+around_20021421.html,"When she said, ""Don't waste your words, they'r..."
1577,Bob Dylan,A Satisfied Mind,/b/bob+dylan/a+satisfied+mind_20021501.html,How many times have you heard someone say \nI...
1578,Bob Dylan,Ain't Gonna Grieve,/b/bob+dylan/aint+gonna+grieve_20275479.html,"By Bob Dylan \n \nWell, I ain't a-gonna grie..."
1579,Bob Dylan,All Over You,/b/bob+dylan/all+over+you_20238880.html,"By Bob Dylan \n \nWell, if I had to do it al..."
1580,Bob Dylan,Ballad Of Donald White,/b/bob+dylan/ballad+of+donald+white_20155910.html,"My name is Donald White, you see, \nI stand b..."
1581,Bob Dylan,Ballad Of Hollis Brown,/b/bob+dylan/ballad+of+hollis+brown_20021599.html,Hollis Brown \nHe lived on the outside of tow...
1582,Bob Dylan,Belle Isle,/b/bob+dylan/belle+isle_20021513.html,One evening for pleasure I rambled to view \n...
1583,Bob Dylan,Beyond Here Lies Nothin',/b/bob+dylan/beyond+here+lies+nothin_20837239....,"Oh well, I love you pretty baby \nYou're the ..."
1584,Bob Dylan,Beyond The Horizon,/b/bob+dylan/beyond+the+horizon_20538898.html,"Beyond the horizon, behind the sun \nAt the e..."
1585,Bob Dylan,Black Crow Blues,/b/bob+dylan/black+crow+blues_20021148.html,"I woke in the mornin', wand'rin', \nWasted an..."


In [5]:
# Токенизируем
def tokenizers(text):    
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

In [6]:
#  Для начала делим всю нашу выборку в соотношении 20:80
text_train, text_test, artist_train, artist_test = train_test_split(bands['text'], bands['artist'], test_size=0.2)

In [7]:
# Добавим cтоп-слова
stopwords = stopwords.words('english')

In [13]:
# Векторизация и Байес
res = Pipeline([('bow', CountVectorizer(analyzer=tokenizers, stop_words=stopwords)),
                ('classifier', MultinomialNB())])
res.fit(text_train, artist_train)
artist_pred = res.predict(text_test)
print(classification_report(artist_test, artist_pred))

                  precision    recall  f1-score   support

       Bob Dylan       0.62      0.44      0.52        34
    Donna Summer       0.49      0.61      0.54        31
   George Strait       0.64      0.70      0.67        40
Gordon Lightfoot       0.64      0.62      0.63        47

     avg / total       0.61      0.60      0.60       152



Хуже всего определяется Donna Summer, но её песен больше всего...